In [12]:
from mongoengine import connect
import math
import json
import copy
import sys
sys.path.insert(0, '..\\mongo-db-scripts')
from Bug import Bug

connect('bug_reports_db', host='127.0.0.1', port=27017)

with open('data\\fields_name_not_cf.json') as input:
  fields = json.load(input)

In [21]:
bugs_counting_changes = []
temp_bug_init = { 'reporter': {}, 'others': {}}

for field in fields:
  temp_bug_init['reporter'][field] = 0
  temp_bug_init['others'][field] = 0

def count_changes(bug):
  counted_changes = copy.deepcopy(temp_bug_init)

  for changes in bug['history']:
    if (changes['who'] == bug['creator']):
      user_type = 'reporter'
    else:
      user_type = 'others'

    for change in changes['changes']:
      field_name = change['field_name']

      if (field_name not in fields):
        continue
      
      counted_changes[user_type][field_name] += 1
      
  return counted_changes

In [ ]:
BUGS_LIMIT = 50000
total_bugs = Bug.objects.count()
bugs_per_iteration = math.ceil(total_bugs/BUGS_LIMIT)

for i in range(bugs_per_iteration):
  skip_quantity = i * BUGS_LIMIT
  result = Bug.objects.only('bug_id', 'history', 'creator').limit(BUGS_LIMIT).skip(skip_quantity)

  bugs = [bug.to_mongo().to_dict() for bug in result]

  for bug in bugs:
    temp_bug = count_changes(bug)
    temp_bug['bug_id'] = bug['bug_id']
    bugs_counting_changes.append(temp_bug)

In [23]:
with open('data\\processed_bugs_counting_changes_users_final.json', 'w') as output:
  json.dump(bugs_counting_changes, output)